## Esta notebook es creada para hacer pruebas con el procesamiento de la informacion de los documentos csv

In [70]:
import os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

### _para limpiar los documentos extraidos del pdf_
    ->> obtener una lista de todas las materias con sus nrc y la carrera en la que se ofertan
    --- limpiar la lista de materias (toLower)
    --- sustituir los nombres de materias por los correctos
    ---reemplazar los NAN donde se genera esto

In [9]:
###celda de erorres TODO
data[0][16:22]

,Materia,Clave,Secc,NRC,Dias,Hora,Salon,Profesor,Doc
16,Inteligencia Artificial,CCOM 262,OO1,29089,AJ,0900-1059,1EMA4/409,ZACARIAS - FLORES FERNANDO,0
17,Inteligencia Artificial,CCOM 262,OO1,29089,L,1000-1059,1EMA4/409,NaN,0
18,Arqu. Funcional de,NaN,NaN,-1,NaN,NaN,NaN,ZACARIAS - FLORES FERNANDO,0
19,Computadora,IDCO 200,OO1,29093,L,1300-1359,1CCO5/201,NaN,0
20,Arqu. Funcional de,NaN,NaN,-1,NaN,NaN,NaN,SANCHEZ - GALVEZ MARIA EUGENIA N,0
21,Computadora,IDCO 200,OO1,29093,M,1300-1459,1CCO3/309,NaN,0


In [5]:
#recibe un directorio
#retorna todos los documentos de ese directorio
def getDocsinDir(directory):
    docs=[]
    for root,directories,files in os.walk(directory+'/'):
        for f in files:
            docs.append(directory+'/'+f)
    return docs
#recibe una lista de nombres de documentos
#retorna un lista de dataFrames
def getCSVs(docs):
    data=[]
    for x in docs:
        data.append(pd.read_csv(x))
    return data
#recibe un panda.Dataframes
    #elimina todos los elementos nulos y ajusta los nombres de materias y profesores
#retorna el mismo Dataframe  
def cleanData(data):    
        data = data[data['Materia'].notna()]
        data['NRC']=data['NRC'].replace(to_replace=np.nan,value=-1)
        #data[2].dropna(inplace=True)
        data.reset_index(drop=True,inplace=True)
        bad=data.loc[lambda y: y.Materia=='Materia']
        #print(bad)
        badIndexes=bad.index.to_list()
        #print(badIndexes)
        data.reset_index(drop=True,inplace=True)
        data=data.drop(badIndexes)
        
        return data
#recibe el arreglo de dataframes 
##Recupera todas las materias con sus respectivos nrc
## devuelve un dataFrame de todas la materias con su nrc
def getCleanMaterias(data):
    
    materias=data[0].Materia.tolist()
    nrcs=data[0].NRC.tolist()
    numOfDoc=data[0].Doc.tolist()
    
    for x in range(len(data)-1):
        materias.extend(data[x+1].Materia.tolist())
        nrcs.extend(data[x+1].NRC.tolist())
        numOfDoc.extend(data[x+1].Doc.tolist())
        
    antesMat=""
    antesNrc=0
    realMaterias=[]
    for m,n,d in zip(materias,nrcs,numOfDoc):
        ahoraMat=m
        ahoraNrc=n
        if(antesNrc==-1):
            #print('aqui----'+antesMat+' '+m,n,type(n))
            helpDic={0:antesMat+' '+m,1:n,2:str(d)}
            realMaterias.append(helpDic)
        elif(ahoraNrc!=-1):
            helpDic={0:m,1:n,2:str(d)}
            #print(m,n,type(n))
            realMaterias.append(helpDic)
        antesNrc=n
        antesMat=m
    p=pd.DataFrame(realMaterias)
    p=p.rename(columns={0:'Materia',1:'NRC',2:'Doc'})
    p=p.drop_duplicates()
    return p

In [6]:
docs=getDocsinDir("filesCSV")
data=getCSVs(docs)
for x in range(len(data)):
        data[x]=cleanData(data[x])
        p=[x]*len(data[x])
        data[x]['Doc']=p ##genera una columna que almacene a que numero de documento pertenece
        
Materias=getCleanMaterias(data)
nombresMaterias=Materias.Materia.unique().tolist()


### _para indexador de palabras de documento de texto:_
        → retorna un diccionario con todas las palabras encontradas

In [ ]:
#def simbologia
#???? => es necesario?
# [linea] ? [linea] => cual de las dos opciones es mejor?
#DEFINIR CLASE Match
#     class Match:
#      numerodelinea=int
#      ubicacion en la linea=[]  

#para indexador de indice invertido
    #leer todo el documento linea a linea
    #limpiar el texto
    #TOKENIZAR CADA PALABRA EN LA LINEA 
    #almacenar numero de palabras en la linea ????? (necesario si queremos calcular la pocision de la palabra relativa al doc)
    #si la palabra NO esta en stop words y aun no esta en el diccionario:
        #agregar la palabra al diccionario con su diccionario correspodiente
        #guardar en su diccionario el numero de documento al que corresponde
        #inicializar el primer elemnto de la lista (APARICIONES)=1 de la palabra en el diccionario del documento
        #crear una nueva instancia match
           #almacenar en que linea esta (NUM_LINEA)
            #almacenar la ubicacion de la palabra  (relativo a la linea)
                # almacenar que numero de palabra ocupa en la linea ? almacenar numero de caracter en el que inicia la palabra
        #agregar la instancia de Match a la lista del diccionario de la palabra
    #si la palabra ya esta en el diccionario
        #incrementar por uno (APARICIONES)
        #revisar el match.Linea anterior
            #si es la misma linea en la que está
                #actualizar la lista de apariciones
            #sino
                #crear una nueva instancia match
                #agregar la instancia de Match a la lista del diccionario de la palabra
        
#estructura: lista de diccionarios t.q:

#    {'token o palabra': {NUM_DOC:[APARICIONES,Match1,Match2],NUM_DOC:[APARICIONES,Match1,Match2]}}
# [[,,,,][,,,,]] cada lista corrsponde a  un documento y la pocision de la lista almacena el numero de palabras en la linea

In [124]:
class Match:
    def __init__(self,numeroLinea : int, ocurrencias: list):
        self.linea=numeroLinea
        self.ocurrencias=ocurrencias
    def addOcurrencias(self, new:list):
        for elem in new:
            if elem not in self.ocurrencias:
                self.ocurrencias.append(elem)
    def getOcurrencias(self):
        return self.ocurrencias
    def getLinea(self):
        return self.linea
    def getNumOcurrencias(self):
        return len(self.ocurrencias)

In [169]:
from nltk.tokenize import word_tokenize
import re
from unicodedata import normalize
stopWords=set(stopwords.words('spanish'))
stopWordsEngl=set(stopwords.words('english'))
#recibe una cadena
#retorna una cadena limpia
def normal(string):
    string=re.sub('_',' ',string)
    string=re.sub(r'[^\w\s]+',' ',string) #elimina simbolos
    string=re.sub(r"^\d+\s|\s\d+\s|\s\d+$",' ',string) ##elimina numeros solos
    string=re.sub(r"^\d+\s|\s\d+\s|\s\d+$",' ',string) ##elimina numeros solos
    string=string.lower()#↓↓ RETIRA DIACRITICOS MENOS ñ
   #string = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+",r"\1",normalize( "NFD",string),0,re.I)
    #string = normalize( 'NFC', string)
    return string

normal('ho4lá 5ño 1988´.dfs_?¡¿ 2 10921d aquíHOOL8 7545 :2012. 1901. más') #test

'ho4lá 5ño dfs   10921d aquíhool8    más'

In [221]:
#rutina para indexador
# recibe una lista de nombres de documentos 
#retorna un diccionario (la estructura del indexador)
def makeIndex(docs:list):
    headIndexer={}
    numeroPalabrasPorDoc=[]
    numDoc=0
    for x in docs:
        numeroPalabrasPorDoc.append([])
        tokens=[]
        doc=open(x,encoding='utf8')
        data=doc.readlines()
        ##para cada linea en el documento
        for i in range(len(data)):
            data[i]=normal(data[i])
            lineTokenized=word_tokenize(data[i])
            numeroPalabrasPorDoc[numDoc].append(len(lineTokenized))
            if len(lineTokenized) > 0:
                lineTokenized.insert(0,i) ##almacena el numero de linea al que pertenece
                tokens.append(lineTokenized)
    
        for linea in tokens:
            numpalabra=0
            numlinea=linea[0]
            for palabra in linea:
                if type(palabra) is not int:
                    if palabra not in stopWords and palabra not in stopWordsEngl:
                        if palabra not in headIndexer:
                            match=Match(numlinea,[numpalabra])
                            headIndexer.update({palabra:{numDoc:[1,match]}})## psible bug
                        else:
                            #si el documento no esta en elindexador
                            dicDocs= headIndexer.get(palabra)
                            if numDoc not in dicDocs:
                                dicDocs.update({numDoc:[1,match]})
                            #si el documento ya esta en el indexador
                            else:
                                headIndexer.get(palabra).get(numDoc)[0]+=1
                                matchAnterior=headIndexer.get(palabra).get(numDoc)[-1]
                                if matchAnterior.getLinea() == numlinea:
                                    matchAnterior.addOcurrencias([numpalabra])
                                else:
                                    match=Match(numlinea,[numpalabra])
                                    headIndexer.get(palabra).get(numDoc).append(match)
                    numpalabra+=1
            numlinea+=1
        numDoc+=1
    return headIndexer,tokens

In [228]:
txt=['textDocs/likeAPrayer.txt','textDocs/sencillosMasVendidosDelMundo.txt']
indexado,tokens=makeIndex(txt)
#indexado
#tokens

[[0, 'anexo', 'sencillos', 'más', 'vendidos', 'en', 'el', 'mundo'],
 [1, 'ir', 'a', 'la', 'navegaciónir', 'a', 'la', 'búsqueda'],
 [3,
  'según',
  'el',
  'libro',
  'guinness',
  'de',
  'los',
  'récords',
  'white',
  'christmas',
  'de',
  'bing',
  'crosby',
  'es',
  'el',
  'sencillo',
  'más',
  'vendido',
  'en',
  'el'],
 [4,
  'mundo',
  'con',
  'ventas',
  'estimadas',
  'en',
  'más',
  'de',
  'millones',
  'de',
  'copias',
  'en',
  'la',
  'actualidad',
  'se',
  'han',
  'registrado',
  'más',
  'de',
  'millones'],
 [5,
  'de',
  'copias',
  'vendidas',
  'en',
  'todo',
  'el',
  'mundo',
  'de',
  'la',
  'versión',
  'de',
  'crosby',
  'siendo',
  'por',
  'mucho',
  'el',
  'single',
  'más',
  'vendido',
  'de',
  'la',
  'historia'],
 [7,
  'el',
  'sencillo',
  'de',
  'doble',
  'lado',
  'a',
  'candle',
  'in',
  'the',
  'wind',
  'something',
  'about',
  'the',
  'way',
  'you',
  'look',
  'tonight',
  'de',
  'elton',
  'john',
  'es'],
 [8,
  'cons

In [219]:
def searchIndex(word,index):
    buffer=[]
    for x in txt:
        buffer.append(open(x,encoding='utf8'))
    hbuffer=[]
    for x in buffer:
        hbuffer.append(x.readlines())
        
    if word in index:
        dicDoc=index.get(word)
        for doc in dicDoc:
            #print(documentos[doc])
            for value in dicDoc[doc]:
                if type(value) is int:
                    print("{} ocurrencias en {}".format(value,txt[doc]))
                else:
                    lst=list(map(lambda x: x+1,value.getOcurrencias()))
                    print("\tEn linea {} \n\t\tpalabra {}".format(value.getLinea()+1,lst))
                    print("\n\t{}".format(hbuffer[doc][value.getLinea()]))
    else:
        return None

In [230]:
searchIndex('hombre',indexado)

1 ocurrencias en textDocs/likeAPrayer.txt
	En linea 17 
		palabra [8]

	blancos. Cuando arrestan por error a un hombre negro por el crimen, se esconde en una iglesia para pedir fortaleza y 



In [151]:
match=Match(9,[1,2])
listadelistas=[]
listadelistas.append([])
listadelistas[0].append(0)
listadelistas.append([])
dic={'hola':{'h':[11,12,14],'o':[45,65,900],'l':[4,9,0,88]},'pero':{'p':[1,4,6] }}

#dic.get('hola').get('o')[0]+=10
#p=nd.get('o')
#p.extend([5,4])
#dic.get('hola').get('o').append(12)
#pp=dic.get('hola').get('o')[-2]
#pp.addOcurrencias([3,4])
#dic.get('hola').get('h')[-1].getOcurrencias()
#r =dic.get('hola')
#r.update({'p':[1]})
'más' in stopWords

True

	 hola papa :[1, 2, 3, 4, 5]
